In [3]:
import yaml
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm

In [4]:
with open('api_key.yaml') as f :
    API_KEY = yaml.load(f, yaml.FullLoader)['google_api']

In [37]:
books = pd.read_csv('../data/Ratings.csv')

In [38]:
books = books.groupby('ISBN', as_index=False).agg({'Book-Rating' : 'count'})

In [39]:
book_ISBNs = list(books[books['Book-Rating'] >= 5]['ISBN'])

In [41]:
for i in range(len(book_ISBNs)) :
    if len(book_ISBNs[i]) != 10 :
        print(book_ISBNs[i], end=' ')
        book_ISBNs[i] = book_ISBNs[i].replace(' ', '')
        book_ISBNs[i] = book_ISBNs[i].replace('/', '')
        book_ISBNs[i] = book_ISBNs[i].replace('.', '')

00000000 000000000 00000000000 000000000000 0000000000000 0006392172PBK 000714415 006101351 009177925 00977319 022406103 030371706 033031582 0345247191125 057113747400 059035342 064400077 064400085 06440031X 067697309 067976402 071831006998 080411868 088061430420 08881832240 10987654321 11204982 11238356 11254041 12123546500 13579108642 15916847 1592825X 17224918 22401269 22530152730 344235353 345245504195 357910864 380976587 381050723 399501487 55307200175 5601640090335 71908979 8014366020420 8022264188142 840000001 843397023 848997943 857237528 880447679 880611587 8807081563X 880781385 88741800047 887641272 899792145 905513369 9770104987002 9770390107900 9771129085131 9771129085155 9782020108928 9788877682208 987932504 M79702002 

In [28]:
def get_data(*keys):
    data = volume_info
    global err_toggle
    try :
        for key in keys:
            data = data[key]
        return data
    except :
        err_toggle = 1
        return None

In [46]:
book_infos = pd.DataFrame(columns=['ISBN', 'ISBN13', 'description', 'page_cnt', 'categories', 'average_rating', 'rating_count', 'thumbnail', 'synopsis'])

In [48]:
row_num = 0
no_data_cnt = 0
err_cnt = 0

for i in tqdm(range(len(book_ISBNs))) :
    
    ISBN = book_ISBNs[i]
    err_toggle = 0
    
    response = requests.get(f"https://www.googleapis.com/books/v1/volumes?q={ISBN}")
    
    try : volume_info = response.json()['items'][0]['volumeInfo']
    except :
        no_data_cnt += 1
        book_infos.loc[row_num] = [ISBN, None, None, None, None, None, None, None, None]
        row_num += 1
        continue   
    
    try :
        if volume_info['industryIdentifiers'][0]["identifier"] != ISBN:
            no_data_cnt += 1
            book_infos.loc[row_num] = [ISBN, None, None, None, None, None, None, None, None]
            row_num += 1
            continue
    except :
        no_data_cnt += 1
        book_infos.loc[row_num] = [ISBN, None, None, None, None, None, None, None, None]
        continue
        
    
    description = get_data('description')
    ISBN_13 = get_data('industryIdentifiers', 1, "identifier")
    page_cnt = get_data('pageCount')
    categories = get_data('categories')
    average_rating = get_data('averageRating')
    rating_count = get_data('ratingsCount')
    thumbnail = get_data('imageLinks', 'thumbnail')
    info_link = get_data('infoLink')
    
    if info_link != None : 
        html = BeautifulSoup(requests.get(info_link).text)
        try : synopsis = html.select_one('#synopsistext').text
        except : synopsis = None
        
    if err_toggle == 1 :
        err_cnt += 1
    
    book_infos.loc[row_num] = [ISBN, ISBN_13, description, page_cnt, categories, average_rating, rating_count, thumbnail, synopsis]
    
    row_num += 1

  0%|          | 0/43765 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
  3%|▎         | 1465/43765 [26:36<11:42:56,  1.00it/s]/opt/conda/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
  4%|▍         | 1824/43765 [33:40<14:39:43,  1.26s/it]/opt/conda/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ra